In [25]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

### MNIST con K-vecinos más vercanos
En esta segunda sesión práctica vamos a entrenar clasificadores basados en distancias.


In [27]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)



/usr/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [ ]:
# Partición de los datos
tam_test = 0.1
tam_val = 0.1
tam_train = 0.8

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=tam_test, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=tam_val/(tam_train + tam_test), random_state=23) 

### **Ejercicio 2**   

Prueba un clasificador KNeighborsClassifier

In [ ]:
## Prueba el clasificador KNeighborsClassifier
clf = KNeighborsClassifier().fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

### **Ejercicio 3**   

Encuentra el mejor valor de "K" con el conjunto de validación y obtén el resultados para test.

In [ ]:
## Estimar el mejor parámetro K con el conjunto de validación
best = -1
bestK = -1

for k in [1,2,3,4,5]:
    clf = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    acc = accuracy_score(y_val, clf.predict(X_val))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

    if acc > best:
        best = acc
        bestK = k

In [ ]:
## Obtener tasa de acierto con el conjunto de test para el mejor K
X = np.concatenate((X_train, X_val))
Y = np.concatenate((y_train, y_val))

clf = KNeighborsClassifier(n_neighbors=bestK).fit(X, Y)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

## **Ejercicio 4**

Cambia la distancia que se emplea (por defecto la L2) por la L1. Ver la documentación para ver cómo poder definir funciones de distancia y pasarlas como parámetro. Esto ralentiza bastante la clasificación pero se puede emplear el parámetro "n_jobs" para paralelizar el proceso. Aún así el resultado es muy lento. Podríamos concluir que sk-learn no es una librería adecuada para clasificar con KNN con métricas definidas por el usuario. Por ejemplo no permite definir un algoritmo rápido de tipo kd_tree.

In [ ]:
## Modifica la función de distancia

clf = KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=-1).fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

'''
X = np.concatenate((X_train, X_val))
Y = np.concatenate((y_train, y_val))

clf = KNeighborsClassifier(n_neighbors=3, p=1, n_jobs=-1).fit(X, Y)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))
'''

**Ejercicio 5**

Encuentra el mejor valor de "K" y la mejor proyección a PCA con el conjunto 

In [ ]:
max = -1
bestNc = -1
bestK = -1

for nc in [2, 4, 8, 16, 32]:
    for k in [1,2,3,4,5]:
        clf = make_pipeline(PCA(n_components=nc), KNeighborsClassifier(n_neighbors=k, p=1, n_jobs=-1)).fit(X_train, y_train)
        acc = accuracy_score(y_val, clf.predict(X_val))
        print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

        if acc>max:
            max = acc
            bestNc = nc
            bestK = k


In [ ]:
def midist(x, y):
    return np.sum(np.abs(x-y))

In [ ]:
midist(np.array([1,2,3]),np.array([4,5,6]))

In [ ]:
clf = KNeighborsClassifier(n_neighbors=3, metric=midist, n_jobs=-1).fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

In [28]:
clf = KNeighborsClassifier(n_neighbors=3, metric=midist, n_jobs=-1).fit(X_train, y_train)
acc = accuracy_score(y_val[:100], clf.predict(X_val[:100,:])[:100])
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de KNeighborsClassifier(metric=<function midist at 0x7f526d8c9120>, n_jobs=-1,
                     n_neighbors=3) es 95.0%
